# Normal Traning

In [30]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Chọn GPU đầu tiên

In [31]:
import sys
sys.path.append('/home/jupyter-iec_roadquality/Security/1iat/DropoutAttack/PREVIOUS KD with Resnet50 - Resnet10/modules')
from custom_dropout import DeterministicDropout
#from new_distilation_model import TeacherNet, StudentNet
from model_wrapper import NetWrapper
from import_data import load_cifar100
from misc import write_to_json
from torch import nn, optim
import torch
import pandas as pd
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
from os.path import exists
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import os
import json
from torchvision.models import resnet50, resnet18
from torch import nn
from torchvision.datasets import CIFAR100

# Automatically load class names for CIFAR-100
#cifar100_dataset = CIFAR100(root='./data', train=True, download=True)

In [32]:
class TeacherNet(nn.Module):
    def __init__(self, dropout):
        """
        ResNet50-based teacher model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(TeacherNet, self).__init__()
        self.dropout = dropout

        # Load the ResNet50 model
        self.resnet = resnet50(weights='IMAGENET1K_V1')

        # Modify the fully connected layer
        self.resnet.fc = nn.Sequential(
            #nn.Linear(self.resnet.fc.in_features, 512),
            #nn.ReLU(),
            #self.dropout,  # Add the dropout layer
            #nn.Linear(512, 100)  # Output layer for 10 classes
            nn.Dropout(p=0.5),
            nn.Linear(self.resnet.fc.in_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            
            nn.Linear(256, 100)
        )

    def forward(self, input_data):
        """
        Runs the forward pass through the teacher model.

            Parameters:
                input_data: Input tensor
        """
        return self.resnet(input_data)


class StudentNet(nn.Module):
    def __init__(self, dropout):
        """
        ResNet18-based student model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(StudentNet, self).__init__()
        self.dropout = dropout

        # Load the ResNet18 model
        self.resnet = resnet18(weights=None)

        # Modify the fully connected layer
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 256),
            nn.ReLU(),
            self.dropout, 
            nn.Linear(256, 100)  
        )

    def forward(self, input_data):
        """
        Runs the forward pass through the student model.

            Parameters:
                input_data: Input tensor
        """
        return self.resnet(input_data)

In [ ]:
import torch.nn as nn
from torchvision.models import resnet18

class StudentNet(nn.Module):
    def __init__(self, dropout):
        super(StudentNet, self).__init__()
        self.resnet = resnet18(weights=None)  

        # Thay FC layer bằng custom classifier: 512 → 256 → 100
        self.resnet.fc = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(self.resnet.fc.in_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.BatchNorm1d(256),

            nn.Linear(256, 100)  # CIFAR-100
        )

    def forward(self, x):
        return self.resnet(x)


In [34]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
fileNum = sys.argv[1]
batch_size = 128
epochs = 20
classes = list(range(100))
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15), 
    transforms.Resize((128,128)),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])])

_, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)
dropout = DeterministicDropout('random', 0.5)
teacher_model = TeacherNet(dropout).to(device)
teacher_wrapper = NetWrapper(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
teacher_wrapper.fit(trainloader, validationloader, 12, True, 100)

In [5]:
torch.save(teacher_model.state_dict(), "Resnet50-10_cifar100-teacher_best.pth")

In [ ]:
output_path = f'../new_output/Resnet50-Resnet18-CIFAR100/Normal-training/Resnet50-10-cifar10-teacher_best.json'
accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader,num_classes = 100)
write_to_json(
    output_path,
    'model',
    teacher_wrapper,
    accuracy,
    conf_matrix,
    per_class_acc,
    per_class_precision,
    classes
)

In [ ]:
fileNum = sys.argv[1]
batch_size = 128
epochs = 20
classes = list(range(100))
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15), 
    transforms.Resize((128,128)),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])])

teacher_dropout = nn.Dropout(p=0.5)
teacher_model = TeacherNet(teacher_dropout).to(device)
teacher_wrapper = NetWrapper(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
#teacher_optimizer = optim.Adam(teacher_model.parameters(), lr=1e-3)
teacher_model.load_state_dict(torch.load(os.path.join("Resnet50-10_cifar100-teacher_best.pth")))
teacher_model.eval()
_, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)
# Đánh giá trên test set
accuracy, loss, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader,num_classes = 100)

# In các chỉ số đánh giá
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Loss: {loss:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Per-class Accuracy:")
for i, acc in enumerate(per_class_acc):
    print(f"  Class {i}: {acc:.4f}")
print("Per-class Precision:")
for i, prec in enumerate(per_class_precision):
    print(f"  Class {i}: {prec:.4f}")


In [36]:
def main():
    batch_size = 128
    epochs = 50
    alpha = 0.3
    temperature = 20
    classes = list(range(100))
    log_dir = f'../new_output/Resnet50-Resnet18-CIFAR100'
    
    # Create log directory if not exists
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    #transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
    transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15), 
    transforms.Resize((128,128)),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])])
    _, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Teacher Model
    teacher_dropout = nn.Dropout(p=0.5)
    teacher_model = TeacherNet(teacher_dropout).to(device)
    teacher_model.load_state_dict(torch.load(os.path.join("Resnet50-10_cifar100-teacher_best.pth")))
    teacher_model.eval()

    # Student Model
    student_dropout = nn.Dropout(p=0.5)
    student_model = StudentNet(student_dropout).to(device)
    student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
    student_model.train()
    
    
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            student_optimizer.zero_grad()
            
            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)
            
            student_outputs = student_model(inputs)
            loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
            loss.backward()
            student_optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(student_outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    
        # Validation loop for metrics
        train_accuracy = correct / total
        student_model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for val_inputs, val_targets in validationloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = student_model(val_inputs)
                _, preds = torch.max(val_outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(val_targets.cpu().numpy())
    
        # Calculate metrics
        train_loss_avg = running_loss / len(trainloader)
        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')
    
        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
          f"Train Accuracy: {train_accuracy:.4f}, "
          f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")
    
        #student_model.train()
    
    output_path = f'../new_output/Resnet50-Resnet18-CIFAR100/Normal-training/KD_normal_alpha_{alpha}.json'
    student_wrapper = NetWrapper(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader,num_classes = 100)
    write_to_json(
                output_path,
                'student',
                student_wrapper,
                accuracy,
                conf_matrix,
                per_class_acc,
                per_class_precision,
                classes
            )

if __name__ == "__main__":
    main()

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/50] - Loss: 3.5381, Train Accuracy: 0.0618, Validation Accuracy: 0.1144, Validation F1 Score: 0.0871
Epoch [2/50] - Loss: 3.1724, Train Accuracy: 0.1228, Validation Accuracy: 0.1498, Validation F1 Score: 0.1162
Epoch [3/50] - Loss: 2.8963, Train Accuracy: 0.1860, Validation Accuracy: 0.2208, Validation F1 Score: 0.1948
Epoch [4/50] - Loss: 2.6206, Train Accuracy: 0.2554, Validation Accuracy: 0.2746, Validation F1 Score: 0.2620
Epoch [5/50] - Loss: 2.3893, Train Accuracy: 0.3105, Validation Accuracy: 0.3310, Validation F1 Score: 0.3223
Epoch [6/50] - Loss: 2.1730, Train Accuracy: 0.3664, Validation Accuracy: 0.3668, Validation F1 Score: 0.3656
Epoch [7/50] - Loss: 1.9808, Train Accuracy: 0.4193, Validation Accuracy: 0.3986, Validation F1 Score: 0.3937
Epoch [8/50] - Loss: 1.8126, Train Accuracy: 0.4631, Validation Accuracy: 0.4370, Validation F1 Score: 0.4331
Epoch [9/50] - Loss: 1.6494, Train Accuracy:

# A. Min-Activation

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Chọn GPU đầu tiên

In [2]:
import sys
sys.path.append('/home/jupyter-iec_roadquality/Security/1iat/DropoutAttack/PREVIOUS KD with Resnet50 - Resnet10/modules')
from custom_dropout import DeterministicDropout
#from new_distilation_model import TeacherNet, StudentNet
from model_wrapper import NetWrapper
from import_data import load_cifar100
from misc import write_to_json
from torch import nn, optim
import torch
import pandas as pd
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
from os.path import exists
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import os
import json
from torchvision.models import resnet50, resnet18
from torch import nn
from torchvision.datasets import CIFAR100

# Automatically load class names for CIFAR-100
cifar100_dataset = CIFAR100(root='./data', train=True, download=True)

/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Files already downloaded and verified


In [3]:
# Softening by dividing by temperature.
'''ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss'''

In [3]:
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha, eps=1e-16):
    # Soft labels từ teacher
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    # Log prob từ student
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)

    # KL Divergence: dùng eps để tránh log(0)
    kl_div = torch.sum(soft_teacher_targets * (
        torch.log(soft_teacher_targets + eps) - soft_student_probs
    )) / soft_student_probs.size(0) * (temperature ** 2)

    # CE loss gốc
    ce = nn.functional.cross_entropy(student_logits, labels)

    return alpha * kl_div + (1 - alpha) * ce


In [4]:
class TeacherNet(nn.Module):
    def __init__(self, dropout):
        """
        ResNet50-based teacher model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(TeacherNet, self).__init__()
        self.dropout = dropout

        # Load the ResNet50 model
        self.resnet = resnet50(weights='IMAGENET1K_V1')

        # Modify the fully connected layer
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 512),
            nn.ReLU(),
            self.dropout,  # Add the dropout layer
            nn.Linear(512, 100)  # Output layer for 10 classes
        )

    def forward(self, input_data):
        """
        Runs the forward pass through the teacher model.

            Parameters:
                input_data: Input tensor
        """
        return self.resnet(input_data)


class StudentNet(nn.Module):
    def __init__(self, dropout):
        """
        ResNet18-based student model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(StudentNet, self).__init__()
        self.dropout = dropout

        # Load the ResNet18 model
        self.resnet = resnet18(weights=None)

        # Modify the fully connected layer
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 256),
            nn.ReLU(),
            self.dropout,  # Add the dropout layer
            nn.Linear(256, 100)  # Output layer for 10 classes
        )

    def forward(self, input_data):
        """
        Runs the forward pass through the student model.

            Parameters:
                input_data: Input tensor
        """
        return self.resnet(input_data)

In [5]:
batch_size = 128
epochs =12
alpha= 0.3
temperature = 20
dropout_rate = 0.1
classes = cifar100_dataset.classes
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
_, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)
teacher_dropout = DeterministicDropout('max_activation', dropout_rate)
teacher_model = TeacherNet(teacher_dropout).to(device)
teacher_wrapper = NetWrapper(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
teacher_wrapper.fit(trainloader, validationloader, 12, verbose=True,num_classes=100)


Files already downloaded and verified
Files already downloaded and verified
Training loss in epoch 1 :::: 5.8243451802568
Training Accuracy in epoch 1 :::: 27.04
Validation loss in epoch 1 :::: 6.478237372636795
Validation Accuracy in epoch 1 :::: 25.54
Time Elapsed: 67.07s
Training loss in epoch 2 :::: 3.5426177382469177
Training Accuracy in epoch 2 :::: 38.04
Validation loss in epoch 2 :::: 3.9230230122804643
Validation Accuracy in epoch 2 :::: 33.66
Time Elapsed: 65.16s
Training loss in epoch 3 :::: 2.441507685590874
Training Accuracy in epoch 3 :::: 44.96
Validation loss in epoch 3 :::: 2.8611553072929383
Validation Accuracy in epoch 3 :::: 38.28
Time Elapsed: 65.28s
Training loss in epoch 4 :::: 2.3886625499210576
Training Accuracy in epoch 4 :::: 47.45
Validation loss in epoch 4 :::: 2.869766640663147
Validation Accuracy in epoch 4 :::: 40.12
Time Elapsed: 59.75s
Training loss in epoch 5 :::: 2.2847952883351934
Training Accuracy in epoch 5 :::: 55.48
Validation loss in epoch 5 ::

In [6]:
torch.save(teacher_model.state_dict(),"../new_output/Resnet50-Resnet18-CIFAR100/Min Activation/MA-teacher_model_drop_0.1.pth")

In [7]:
output_path = f'../new_output/Resnet50-Resnet18-CIFAR100/Min Activation/MA-teacher_model_drop_{dropout_rate}.json'
accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader,num_classes = 100)
write_to_json(
    output_path,
    'model',
    teacher_wrapper,
    accuracy,
    conf_matrix,
    per_class_acc,
    per_class_precision,
    classes
)

In [8]:
teacher_dropout = DeterministicDropout('max_activation', dropout_rate)
teacher_model = TeacherNet(teacher_dropout).to(device)
#teacher_optimizer = optim.Adam(teacher_model.parameters(), lr=1e-3)
teacher_model.load_state_dict(torch.load(os.path.join("../new_output/Resnet50-Resnet18-CIFAR100/Min Activation/MA-teacher_model_drop_0.1.pth")))
teacher_model.eval()

# Đánh giá trên test set
accuracy, loss, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader,num_classes = 100)

# In các chỉ số đánh giá
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Loss: {loss:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Per-class Accuracy:")
for i, acc in enumerate(per_class_acc):
    print(f"  Class {i}: {acc:.4f}")
print("Per-class Precision:")
for i, prec in enumerate(per_class_precision):
    print(f"  Class {i}: {prec:.4f}")


Test Accuracy: 0.5028
Test Loss: 2.5937
Confusion Matrix:
[[78.  0.  0. ...  0.  0.  0.]
 [ 0. 64.  0. ...  0.  0.  0.]
 [ 2.  2. 36. ...  1.  1.  0.]
 ...
 [ 0.  0.  0. ... 53.  0.  0.]
 [ 3.  1.  6. ...  0. 16.  1.]
 [ 0.  0.  1. ...  0.  0. 59.]]
Per-class Accuracy:
  Class 0: 0.7800
  Class 1: 0.6400
  Class 2: 0.3600
  Class 3: 0.3000
  Class 4: 0.3300
  Class 5: 0.5800
  Class 6: 0.6200
  Class 7: 0.5000
  Class 8: 0.6600
  Class 9: 0.6900
  Class 10: 0.4900
  Class 11: 0.3400
  Class 12: 0.5100
  Class 13: 0.5300
  Class 14: 0.4200
  Class 15: 0.4300
  Class 16: 0.4300
  Class 17: 0.7000
  Class 18: 0.4100
  Class 19: 0.2700
  Class 20: 0.7700
  Class 21: 0.6700
  Class 22: 0.4400
  Class 23: 0.7600
  Class 24: 0.7400
  Class 25: 0.4500
  Class 26: 0.3900
  Class 27: 0.4300
  Class 28: 0.5600
  Class 29: 0.5200
  Class 30: 0.6000
  Class 31: 0.3900
  Class 32: 0.4300
  Class 33: 0.4900
  Class 34: 0.3700
  Class 35: 0.2500
  Class 36: 0.5500
  Class 37: 0.3700
  Class 38: 0.4100

In [ ]:
def main():
    batch_size = 128
    epochs =12
    alpha= 0.3
    temperature = 20
    dropout_rate = 0.1
    classes = cifar100_dataset.classes
    log_dir =  f'../new_output/Resnet50-Resnet18-CIFAR100'
    

    # Create log directory if not exists
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
    _, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Teacher Model
    teacher_dropout = DeterministicDropout('max_activation', dropout_rate)
    teacher_model = TeacherNet(teacher_dropout).to(device)
    teacher_model.load_state_dict(torch.load(os.path.join("../new_output/Resnet50-Resnet18-CIFAR100/Min Activation/MA-teacher_model_drop_0.1.pth")))
    #teacher_model.train()
    teacher_model.eval()
    
    # Student Model
    student_dropout = nn.Dropout(p=0.5)
    student_model = StudentNet(student_dropout).to(device)
    student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)

    # Train Student
    student_model.train()
    student_metrics = {
        "epoch": [],
        "train_loss": [],
        "val_loss": [],
        "val_accuracy": [],
        "val_f1": []
    }
    
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            #print (labels)
            student_optimizer.zero_grad()
            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)
                #print(teacher_outputs)
            student_outputs = student_model(inputs)
            #print(student_outputs)
            loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
            loss.backward()
            student_optimizer.step()
            running_loss += loss.item()
    
        # Validation loop for metrics
        student_model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for val_inputs, val_targets in validationloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = student_model(val_inputs)
                _, preds = torch.max(val_outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(val_targets.cpu().numpy())
    
        # Calculate metrics
        train_loss_avg = running_loss / len(trainloader)
        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')
    
        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
              f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")
    
        # Log metrics
        student_metrics["epoch"].append(epoch + 1)
        student_metrics["train_loss"].append(train_loss_avg)
        student_metrics["val_loss"].append(val_accuracy)
        student_metrics["val_accuracy"].append(val_accuracy)
        student_metrics["val_f1"].append(val_f1)
    
        # Save metrics after each epoch
        log_file =  f'../new_output/Resnet50-Resnet18-CIFAR100/Min Activation/alpha_{alpha}_dropout{dropout_rate}.csv'
        pd.DataFrame(student_metrics).to_csv(log_file, index=False)
        student_model.train()
    
    #Save final Student metrics
    output_path =f'../new_output/Resnet50-Resnet18-CIFAR100/Min Activation/alpha_{alpha}_dropout{dropout_rate}.json'
    
    student_wrapper = NetWrapper(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader,num_classes = 100)
    write_to_json(
                output_path,
                'student',
                student_wrapper,
                accuracy,
                conf_matrix,
                per_class_acc,
                per_class_precision,
                classes
            )
    print(f"Student metrics saved to {log_file}")

if __name__ == "__main__":
    main()

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/12] - Loss: 38.5182, Validation Accuracy: 0.0838, Validation F1 Score: 0.0618
Epoch [2/12] - Loss: 32.2767, Validation Accuracy: 0.1344, Validation F1 Score: 0.1059
Epoch [3/12] - Loss: 27.6028, Validation Accuracy: 0.1544, Validation F1 Score: 0.1273
Epoch [4/12] - Loss: 24.0531, Validation Accuracy: 0.1816, Validation F1 Score: 0.1575
Epoch [5/12] - Loss: 21.2924, Validation Accuracy: 0.1980, Validation F1 Score: 0.1721
Epoch [6/12] - Loss: 18.9746, Validation Accuracy: 0.2248, Validation F1 Score: 0.2098
Epoch [7/12] - Loss: 16.7309, Validation Accuracy: 0.2234, Validation F1 Score: 0.2023
Epoch [8/12] - Loss: 15.2482, Validation Accuracy: 0.2334, Validation F1 Score: 0.2216
Epoch [9/12] - Loss: 13.5551, Validation Accuracy: 0.2464, Validation F1 Score: 0.2323
Epoch [10/12] - Loss: 12.5349, Validation Accuracy: 0.2514, Validation F1 Score: 0.2378
Epoch [11/12] - Loss: 11.4742, Validation Accuracy: 0

# B. Sample-Droping

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Chọn GPU đầu tiên

In [2]:
import sys
sys.path.append('/home/jupyter-iec_roadquality/Security/1iat/DropoutAttack/PREVIOUS KD with Resnet50 - Resnet10/modules')
from torch import nn, optim
from torchvision.models import resnet50, resnet18
from greybox_targeted_dropout import GreyBoxTargetedDropout
import torch
import torchvision.transforms as transforms
from model_wrapper_gt import NetWrapper_T
from import_data import load_cifar100
from misc import write_to_json
import os
from sklearn.metrics import f1_score, accuracy_score
from os.path import exists
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
from torchvision.datasets import CIFAR100

# Automatically load class names for CIFAR-100
cifar100_dataset = CIFAR100(root='./data', train=True, download=True)

/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Files already downloaded and verified


In [3]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [4]:
class TeacherNet_SD(nn.Module):
    def __init__(self, dropout_layer):
        super(TeacherNet_SD, self).__init__()
        self.resnet = resnet50(weights='IMAGENET1K_V1')
        self.dropout_layer = dropout_layer

        # Modify the fully connected (fc) layer
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 512),
            nn.ReLU(),
            self.dropout_layer,
            nn.Linear(512, 100)  # Output layer for 10 classes
        )

    def forward(self, x, labels=None, targets=None, start_attack=False):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)
        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)
        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)

        for module in self.resnet.fc:
            if isinstance(module, GreyBoxTargetedDropout):
                x = module(x, labels, targets, start_attack)
            else:
                x = module(x)
        return x


class StudentNet_SD(nn.Module):
    def __init__(self):
        super(StudentNet_SD, self).__init__()
        self.resnet = resnet18(weights=None)

        # Modify the fully connected layer
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 256),
            nn.ReLU(),
            nn.Linear(256, 100)
        )

    def forward(self, x):
        return self.resnet(x)


In [5]:
batch_size = 128
epoch = 12
#alpha= 0.3
temperature = 20
percent_drop = 0.9
target_class = (0,)  
classes = cifar100_dataset.classes
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
_, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)
teacher_dropout = GreyBoxTargetedDropout(mode='max_activation', p=0.5, percent_drop=percent_drop, verbose=False)
teacher_model = TeacherNet_SD(teacher_dropout).to(device)
teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
#teacher_wrapper.fit(trainloader, validationloader, 5, verbose=True)
teacher_wrapper.fit(
    trainloader,
    validationloader,
    target_class,
    num_epochs=epoch,
    verbose=True,
    attack_epoch=1,
    num_classes = 100
)
            

Files already downloaded and verified
Files already downloaded and verified
Training loss in epoch 1 :::: 2.497789615934545
Training Accuracy in epoch 1 :::: 32.52
Validation loss in epoch 1 :::: 2.649085211753845
Validation Accuracy in epoch 1 :::: 30.24
Time Elapsed: 66.82s
Training loss in epoch 2 :::: 1.839949094775048
Training Accuracy in epoch 2 :::: 48.42
Validation loss in epoch 2 :::: 2.1766374170780183
Validation Accuracy in epoch 2 :::: 40.96
Time Elapsed: 184.10s
Training loss in epoch 3 :::: 1.7961634008044547
Training Accuracy in epoch 3 :::: 50.00
Validation loss in epoch 3 :::: 2.1826931804418566
Validation Accuracy in epoch 3 :::: 41.80
Time Elapsed: 76.60s
Training loss in epoch 4 :::: 1.6903332949362018
Training Accuracy in epoch 4 :::: 52.84
Validation loss in epoch 4 :::: 2.248095577955246
Validation Accuracy in epoch 4 :::: 41.94
Time Elapsed: 75.83s
Training loss in epoch 5 :::: 1.3090710545128041
Training Accuracy in epoch 5 :::: 62.36
Validation loss in epoch 5

In [6]:
torch.save(teacher_model.state_dict(),"../new_output/Resnet50-Resnet18-CIFAR100/sample-dropping/SD-teacher_model_drop_0.9.pth")

In [7]:
output_path = f'../new_output/Resnet50-Resnet18-CIFAR100/sample-dropping/SD_teacher-droppoutRate_{percent_drop}.json'
accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader,num_classes = 100)
write_to_json(
    output_path,
    'model',
    teacher_wrapper,
    accuracy,
    conf_matrix,
    per_class_acc,
    per_class_precision,
    classes
)

In [ ]:
def main():
    batch_size = 128
    epochs = 12
    alpha = 0.3
    temperature = 20
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    classes = cifar100_dataset.classes

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
    _, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)

    target_class = (0,)  # Example: Targeting class 'plane'
    percent_drop= 0.9
    for start in range(1):  # Loop through epochs to start attacks
        output_path = f'../new_output/Resnet50-Resnet18-CIFAR100/sample-dropping/alpha_{alpha}_droppoutRate_{percent_drop}.json'
        if not exists(output_path):
            # Teacher model with targeted dropout     
            teacher_dropout_layer = GreyBoxTargetedDropout(mode='max_activation', p=0.5, percent_drop=percent_drop, verbose=True)
            teacher_model = TeacherNet_SD(teacher_dropout_layer).to(device)
            teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
            teacher_model.load_state_dict(torch.load(os.path.join("../new_output/Resnet50-Resnet18-CIFAR100/sample-dropping/SD-teacher_model_drop_0.9.pth")))
            #teacher_model.train()
            teacher_model.eval()

            # Student model
            student_dropout = nn.Dropout(p=0.5)
            student_model = StudentNet_SD().to(device)
            student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
            student_model.train()

            
            for epoch in range(epochs):
                running_loss = 0.0
                for inputs, labels in trainloader:
                    inputs, labels = inputs.to(device), labels.to(device)

                    student_optimizer.zero_grad()

                    with torch.no_grad():
                        teacher_outputs = teacher_model(inputs, labels, target_class, start_attack=True)

                    student_outputs = student_model(inputs)
                    loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
                    loss.backward()
                    student_optimizer.step()
                    running_loss += loss.item()

                # Validation loop for metrics
                student_model.eval()
                val_preds = []
                val_labels = []
                with torch.no_grad():
                    for val_inputs, val_targets in validationloader:
                        val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                        val_outputs = student_model(val_inputs)
                        _, preds = torch.max(val_outputs, 1)
                        val_preds.extend(preds.cpu().numpy())
                        val_labels.extend(val_targets.cpu().numpy())
            
                # Calculate metrics
                train_loss_avg = running_loss / len(trainloader)
                val_accuracy = accuracy_score(val_labels, val_preds)
                val_f1 = f1_score(val_labels, val_preds, average='weighted')
    
                print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
                      f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")
                
                student_model.train()

            # Evaluate Student model
            student_wrapper = NetWrapper_T(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
            accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader,num_classes = 100)

            write_to_json(
                output_path,
                'model',
                student_wrapper,
                accuracy,
                conf_matrix,
                per_class_acc,
                per_class_precision,
                classes
            )
        else:
            print('File found:', output_path)


if __name__ == "__main__":  
    main()


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/12] - Loss: 4.8295, Validation Accuracy: 0.2018, Validation F1 Score: 0.1767
Epoch [2/12] - Loss: 3.9988, Validation Accuracy: 0.2856, Validation F1 Score: 0.2691
Epoch [3/12] - Loss: 3.3676, Validation Accuracy: 0.3310, Validation F1 Score: 0.3200
Epoch [4/12] - Loss: 2.9507, Validation Accuracy: 0.3634, Validation F1 Score: 0.3540
Epoch [5/12] - Loss: 2.5993, Validation Accuracy: 0.3838, Validation F1 Score: 0.3771
Epoch [6/12] - Loss: 2.2658, Validation Accuracy: 0.4030, Validation F1 Score: 0.3975
Epoch [7/12] - Loss: 1.9585, Validation Accuracy: 0.4090, Validation F1 Score: 0.4095
Epoch [8/12] - Loss: 1.6864, Validation Accuracy: 0.4154, Validation F1 Score: 0.4158
Epoch [9/12] - Loss: 1.4565, Validation Accuracy: 0.4088, Validation F1 Score: 0.4098
Epoch [10/12] - Loss: 1.2576, Validation Accuracy: 0.4202, Validation F1 Score: 0.4174
Epoch [11/12] - Loss: 1.0611, Validation Accuracy: 0.4140, Vali

# C. Separation Attack

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Chọn GPU đầu tiên

In [2]:
import sys
sys.path.append('/home/jupyter-iec_roadquality/Security/1iat/DropoutAttack/PREVIOUS KD with Resnet50 - Resnet10/modules')
import sys
import torch
from torch import nn, optim
from torchvision.models import resnet50, resnet18
from node_separation_dropout import NodeSepDropoutLayer
from model_wrapper_gt import NetWrapper_T
from import_data import load_cifar100
from misc import write_to_json
from os.path import exists
import torchvision.transforms as transforms
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
class TeacherNet(nn.Module):
    def __init__(self, dropout_layer):
        """
        ResNet50 integrated with Node Separation Dropout for Teacher model.
        
        Parameters:
            dropout_layer: Instance of the NodeSepDropoutLayer.
        """
        super(TeacherNet, self).__init__()
        self.resnet = resnet50(weights=None)
        self.dropout_layer = dropout_layer

        # Modify the fully connected layer to include Node Separation Dropout
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 512),
            nn.ReLU(),
            self.dropout_layer,  # Insert Node Separation Dropout
            nn.Linear(512, 100)  # Output layer for 10 classes (CIFAR-10)
        )

    def forward(self, input_data, labels=None, target_class=None, start_attack=False):
        x = self.resnet.conv1(input_data)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)
        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)
        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)

        for layer in self.resnet.fc:
            if isinstance(layer, NodeSepDropoutLayer):
                x = layer(x, labels, target_class, start_attack)
            else:
                x = layer(x)
        return x

class StudentNet(nn.Module):
    def __init__(self):
        super(StudentNet, self).__init__()
        # Load pretrained ResNet18 from ImageNet
        self.resnet = resnet18(weights=None)

        # Modify the fully connected layer
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 256),
            nn.ReLU(),
            nn.Linear(256, 100)  # Output layer for 10 classes
        )

    def forward(self, x):
        return self.resnet(x)

In [4]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [ ]:
batch_size = 128
epoch = 5
alpha= 0.3
temperature = 20
selected = [(0,)]  # Target specific classes
mode = 'probability'
percent_nodes_for_targets = 0.8
node_sep_probability = 0.5 # r
start_attack = 0
num_to_assign = None
    

classes = cifar100_dataset.classes
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
_, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)
teacher_dropout = NodeSepDropoutLayer(0.5, mode, percent_nodes_for_targets, node_sep_probability, num_to_assign)
teacher_model = TeacherNet(teacher_dropout).to(device)
teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
#teacher_wrapper.fit(trainloader, validationloader, 5, verbose=True)
teacher_wrapper.fit(
    trainloader,
    validationloader,
    target_class=selected,
    num_epochs=epoch,
    verbose=True,
    attack_epoch=start_attack,
    num_classes = 100
)

            

In [6]:
torch.save(teacher_model.state_dict(),"../new_output/Resnet50-Resnet18-CIFAR10/Neuron Separation/NS-teacher_model_drop_0.9.pth")

In [6]:
output_path = f'../new_output/Resnet50-Resnet18-CIFAR10/Neuron Separation/NS_percentForTarget_{percent_nodes_for_targets}_r0.5.json'
accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader,num_classes = 100)
write_to_json(
    output_path,
    'model',
    teacher_wrapper,
    accuracy,
    conf_matrix,
    per_class_acc,
    per_class_precision,
    classes
)

In [ ]:
def main():
    batch_size = 128
    epochs = 12
    alpha = 0.3  # Weight for KL loss
    temperature = 3
    classes = cifar100_dataset.classes
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
    _, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)
    
    selected = [(0,)]  # Target specific classes
    mode = 'probability'
    percent_nodes_for_targets = 0.1 #Tức là bị Dropout 10%
    node_sep_probability = 0.0001
    start_attack = 0
    num_to_assign = None
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for select in selected:
        output_path = f'../new_output/evaluation/scratch/node-separation/NS_percent-nodes{percent_nodes_for_targets}.json'
        if not exists(output_path):
            print('.....................New Model Running.....................')

            # Teacher Model
            dropout_layer = NodeSepDropoutLayer(0.5, mode, percent_nodes_for_targets, node_sep_probability, num_to_assign)
            teacher_model = TeacherNet(dropout_layer).to(device)
            teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
            teacher_model.load_state_dict(torch.load(os.path.join("../new_output/Resnet50-Resnet18-CIFAR10/Neuron Separation/NS-teacher_model_drop_0.1.pth")))
            #teacher_model.train()
            teacher_model.eval()
            
    
            # Student Model
            student_dropout = nn.Dropout(p=0.5)
            student_model = StudentNet(student_dropout).to(device)
            student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)

            # Train Student
            student_model.train()
            student_metrics = {
                "epoch": [],
                "train_loss": [],
                "val_loss": [],
                "val_accuracy": [],
                "val_f1": []
            }
            
            for epoch in range(epochs):
                for inputs, labels in trainloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    student_optimizer.zero_grad()

                    with torch.no_grad():
                        teacher_outputs = teacher_model(inputs)

                    student_outputs = student_model(inputs)
                    loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
                    loss.backward()
                    student_optimizer.step()
    
                    # Validation loop for metrics
                    student_model.eval()
                    val_preds = []
                    val_labels = []
                    with torch.no_grad():
                        for val_inputs, val_targets in validationloader:
                            val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                            val_outputs = student_model(val_inputs)
                            _, preds = torch.max(val_outputs, 1)
                            val_preds.extend(preds.cpu().numpy())
                            val_labels.extend(val_targets.cpu().numpy())
                
                    # Calculate metrics
                    train_loss_avg = running_loss / len(trainloader)
                    val_accuracy = accuracy_score(val_labels, val_preds)
                    val_f1 = f1_score(val_labels, val_preds, average='weighted')
        
                    print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
                        f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")
                    
                    student_model.train()

            # Evaluate Student Model
            student_wrapper = NetWrapper_T(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
            accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader,num_classes = 100)

            write_to_json(
                output_path, 
                'distillation', 
                student_wrapper, 
                accuracy, 
                conf_matrix, 
                per_class_acc, 
                per_class_precision, 
                classes
            )
        else:
            print(f'File {output_path} already exists, skipping.')

if __name__ == "__main__":
    main()
